# Deploy a model to online endpoints using Triton
Learn how to deploy a model using Triton as an online endpoint in Azure Machine Learning.

Triton is multi-framework, open-source software that is optimized for inference. It supports popular machine learning frameworks like TensorFlow, ONNX Runtime, PyTorch, NVIDIA TensorRT, and more. It can be used for your CPU or GPU workloads.

## Prerequisites

* To use Azure Machine Learning, you must have an Azure subscription. If you don't have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning](https://azure.microsoft.com/free/).

* Install and configure the [Python SDK v2](https://learn.microsoft.com/python/api/overview/azure/ai-ml-readme?view=azure-python#install-the-package).

* You must have an Azure resource group, and you (or the service principal you use) must have Contributor access to it.

* You must have an Azure Machine Learning workspace.

* You must have additional Python packages installed for scoring and may install them with the code below. They include:
    * Numpy - An array and numerical computing library 
    * [Triton Inference Server Client](https://github.com/triton-inference-server/client) - Facilitates requests to the Triton Inference Server
    * Pillow - A library for image operations
    * Gevent - A networking library used when connecting to the Triton Server

In [ ]:
%pip install numpy
%pip install tritonclient[http]
%pip install pillow
%pip install gevent

### Please note, for Triton no-code-deployment, testing via local endpoints is currently not supported, so this tutorial will only show how to set up on online endpoint.

## 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

### 1.1 Configure workspace details
To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. 

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

### 1.2 Generate an endpoint name

In [ ]:
import random

endpoint_name = f"endpoint-{random.randint(0, 10000)}"

### 1.3 Get a handle to the workspace

We use these details above in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group,
    workspace_name,
)

## 2. Configure deployment and associated resources

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `ManagedOnlineDeployment` class.

### Key aspects of deployment 
- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment to use for the deployment. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification.
- `code_configuration` - the configuration for the source code and scoring script
    - `path`- Path to the source code directory for scoring the model
    - `scoring_script` - Relative path to the scoring file in the source code directory
- `instance_type` - The VM size to use for the deployment. For the list of supported sizes, see [Managed online endpoints SKU list](https://docs.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list).
- `instance_count` - The number of instances to use for the deployment

### 2.1 Configure online endpoint
`endpoint_name`: The name of the endpoint. It must be unique in the Azure region. Naming rules are defined under [managed online endpoint limits](https://docs.microsoft.com/azure/machine-learning/how-to-manage-quotas#azure-machine-learning-managed-online-endpoints-preview).

`auth_mode` : Use `key` for key-based authentication. Use `aml_token` for Azure Machine Learning token-based authentication. A `key` does not expire, but `aml_token` does expire. 

Optionally, you can add description, tags to your endpoint.

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")

### 2.2 Configure online deployment
A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `ManagedOnlineDeployment` class and define a Model inline.

#### Key aspects of deployment 
- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `instance_type` - The VM size to use for the deployment. For the list of supported sizes, see [Managed online endpoints SKU list](https://docs.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list).
- `instance_count` - The number of instances to use for the deployment

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment, Model

deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=Model(path="./models", type="triton_model"),
    instance_type="Standard_NC6s_v3",
    instance_count=1,
)

## 3. Deploy to Azure

### 3.1 Create the endpoint
Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [ ]:
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

### 3.2 Create the deployment

Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

### 3.3 Set traffic to 100% for deployment

In [ ]:
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 4. Test the endpoint with sample data
This version of the triton server requires pre- and post-image processing. Below we show how to invoke the endpoint with this processing.

### 4.1 Retrieve the scoring URI


In [ ]:
endpoint = ml_client.online_endpoints.get(endpoint_name)
scoring_uri = endpoint.scoring_uri

### 4.2 Retrieve the endpoint auth key

In [ ]:
keys = ml_client.online_endpoints.get_keys(endpoint_name)
auth_key = keys.primary_key

### 4.3 Test the endpoint

The below script imports pre- and post-processing functions from `scoring_utils/prepost.py`. We first test the model/server readiness and then use those functions to convert the image into a triton readable format and issue the scoring request.

In [ ]:
# Test the blue deployment with some sample data
import requests
import gevent.ssl
import numpy as np
import tritonclient.http as tritonhttpclient
from pathlib import Path
import prepost

img_uri = "http://aka.ms/peacock-pic"

# We remove the scheme from the url
url = scoring_uri[8:]

# Initialize client handler
triton_client = tritonhttpclient.InferenceServerClient(
    url=url,
    ssl=True,
    ssl_context_factory=gevent.ssl._create_default_https_context,
)

# Create headers
headers = {}
headers["Authorization"] = f"Bearer {auth_key}"

# Check status of triton server
health_ctx = triton_client.is_server_ready(headers=headers)
print("Is server ready - {}".format(health_ctx))

# Check status of model
model_name = "model_1"
status_ctx = triton_client.is_model_ready(model_name, "1", headers)
print("Is model ready - {}".format(status_ctx))

if Path(img_uri).exists():
    img_content = open(img_uri, "rb").read()
else:
    agent = f"Python Requests/{requests.__version__} (https://github.com/Azure/azureml-examples)"
    img_content = requests.get(img_uri, headers={"User-Agent": agent}).content

img_data = prepost.preprocess(img_content)

# Populate inputs and outputs
input = tritonhttpclient.InferInput("data_0", img_data.shape, "FP32")
input.set_data_from_numpy(img_data)
inputs = [input]
output = tritonhttpclient.InferRequestedOutput("fc6_1")
outputs = [output]

result = triton_client.infer(model_name, inputs, outputs=outputs, headers=headers)
max_label = np.argmax(result.as_numpy("fc6_1"))
label_name = prepost.postprocess(max_label)
print(label_name)

## 5. Managing endpoints and deployments

### 5.1 Get the logs for the new deployment

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, lines=50
)

## 6. Delete resources

### 6.1 Delete the endpoint and underlying deployment

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)